<H1 align="center"> Automated Expense Categorization - Notebook 02: Baseline Model Development



## 1. Setup

In [25]:
import pandas as pd



## 2. Load data

In [26]:
df = pd.read_csv('../data/processed/transactions_long.csv')

## 3. Define featues and target

In [27]:
df.columns.tolist()


['Income',
 'Age',
 'Dependents',
 'Occupation',
 'City_Tier',
 'Rent',
 'Loan_Repayment',
 'Insurance',
 'Desired_Savings_Percentage',
 'Desired_Savings',
 'Disposable_Income',
 'Potential_Savings_Groceries',
 'Potential_Savings_Transport',
 'Potential_Savings_Eating_Out',
 'Potential_Savings_Entertainment',
 'Potential_Savings_Utilities',
 'Potential_Savings_Healthcare',
 'Potential_Savings_Education',
 'Potential_Savings_Miscellaneous',
 'entity_id',
 'category',
 'amount']

In [28]:
# -----------------------------------------------------------
# Step 3.1: Predict Amount for Savings
# -----------------------------------------------------------

y_savings = df["Desired_Savings"]
x_savings = df.drop(columns=["Desired_Savings", "entity_id"])

In [29]:
# -----------------------------------------------------------
# Step 3.2 Predict Overspending
# -----------------------------------------------------------

df["Overspend_Flag"] = (df["amount"] > df["Disposable_Income"]).astype(int)
y_spending = df["Overspend_Flag"]
x_spending = df.drop(columns=["Overspend_Flag", "entity_id", "category"])


In [30]:
# # -----------------------------------------------------------
# # Step 3.3 Predict Spending Category
# # ----------------------------------------------------------
# future_spending = [
    
# ]
# y_future_spending = df[future_spending]
# x_future_soending = pd.get_dummies(df, columns=["category"], drop_first=True)



# 4. Split data into train and test

# 5. Scale

# 6. Baseline models
 - Linear Regression
 - Linear SVM
 - Random Forest

# 7. Evaluate Models

# 8. Results